In [ ]:
import torch
import warnings
import networkx as nx
import matplotlib.pyplot as plt
import scipy
from scipy import io
from pylab import *
from copy import deepcopy
from torch.utils.data import DataLoader
import copy
import torch.nn as nn
import networkx as nx
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score,precision_score,recall_score
from tqdm.notebook import tqdm
import numpy as np
warnings.filterwarnings("ignore")

In [ ]:
mat = scipy.sparse.load_npz('deep_patient_matrix.npz')  ## Adjacency Matrix

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
mat2=mat.nonzero() ## index of rows and columns having edge
mat2 = torch.tensor(mat2).to('cuda')

In [ ]:
## splitting into train and test set
k = 0
prev = 0
sz = 0
psz=0
testpsz=0
testsz=0
mat_train0 = torch.ones(3021969).to('cuda')
mat_train1 = torch.ones(3021969).to('cuda')
mat_test1 = torch.ones(1200000).to('cuda')
mat_test2 = torch.ones(1200000).to('cuda')
for i in tqdm(range(len(mat.nonzero()[0]))):
    if mat2[0][i] == k :
        continue
    k = k+1    
    if i-prev > 50 :
        psz=sz
        sz = sz+i-prev-40
        mat_test1[testsz:testsz+40] = mat2[0][i-40:i]
        mat_test2[testsz:testsz+40] = mat2[1][i-40:i]
        testsz = testsz+40
        mat_train0[psz:sz] = mat2[0][prev:i-40]
        mat_train1[psz:sz] = mat2[1][prev:i-40]
    else :
        psz=sz
        sz = sz+i-prev
        mat_train0[psz:sz] = mat2[0][prev:i]
        mat_train1[psz:sz] = mat2[1][prev:i]
    prev = i

In [ ]:
class Recommender(nn.Module):
    def __init__(self,num_users,num_artists,num_factors):
        super().__init__()
        self.u = nn.Embedding(num_users,num_factors)
        self.a = nn.Embedding(num_artists,num_factors)
        self.u.weight.data.uniform_(-.01,.01)
        self.a.weight.data.uniform_(-.01,.01)
        self.ub = nn.Embedding(num_users,1)
        self.ab = nn.Embedding(num_artists,1)
        self.ub.weight.data.uniform_(-.01,.01)
        self.ab.weight.data.uniform_(-.01,.01)
     
    def forward(self,index1,index2,test = 0):
        us,art = self.u(index1) , self.a(index2)
        dp = (us*art).sum(1)
        dpb = dp+self.ub(index1).squeeze()+self.ab(index2).squeeze()
        if test == 1 :
            return self.u,self.ub,self.a,self.ab
        return dpb

In [ ]:
train_pat = mat_train0[:sz].cpu()  ## indices of patients
train_icd = mat_train1[:sz].cpu()-30000  ## indices of diseases

In [ ]:
train_pat = torch.tensor(train_pat)
train_icd = torch.tensor(train_icd)

In [ ]:
model = Recommender(30000,47364,16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1, momentum=0.9, weight_decay=5e-4)
y = torch.ones(sz,)
y = y.to(device)

In [ ]:
train_pat = train_pat.long()
train_icd = train_icd.long()

In [ ]:
epochs = 200
model = model.to(device)
for epoch in tqdm(range(1, epochs)):
    model.train()
    optimizer.zero_grad()
    train_pat = train_pat.to(device)
    train_icd = train_icd.to(device)
    pred = model(train_pat,train_icd)
    #print(pred.shape)
    loss = torch.sum((y-pred) ** 2)/sz
    loss.backward()
    optimizer.step()
    if epoch %10 ==0 :
        print(loss)

In [ ]:
pat,pat_b,icd,icd_b = model(train_pat,train_icd,test=1)

In [ ]:
pat_idx = torch.ones(30000,).long().to(device)
icd_idx = torch.ones(47364,).long().to(device)
for i in range(30000):
    pat_idx[i] = i
    icd_idx[i] = i
for i in range(30000,47364):
    icd_idx[i] = i

In [ ]:
patient = pat(pat_idx)+pat_b(pat_idx)
icdcode = icd(icd_idx)+icd_b(icd_idx)

In [ ]:
nrecommend = 100
total_recommendable = 3021969
hit = 0
for k in tqdm(range(30)):
    pred = patient[(k)*1000:(k+1)*1000].matmul(icdcode.t())
    sorted, indices = torch.sort(pred)
    for i in range(1000):
       for j in range(nrecommend):
          if mat[i,indices[i][47363-j].item()+30000] == 1 :
            hit=hit+1
precision = hit/(30000*nrecommend)
recall = hit/total_recommendable
print("no of hits = ",hit,"Recall = ", recall,"Precison = ",precision)            

In [ ]:
### prediction for patient 
id = 1 ## id or index no of patient
top_k = 10 ## top k predictions
icd_code = np.load('Models/icd_codes.npy',allow_pickle = True)
pred = patient[id].matmul(icdcodes[30000:].t())
sorted, indices = torch.sort(pred)
for j in range(top_k-1,-1,-1):
   idx = indices[47363-j].item()
   if mat[idx+30000] == 1 :
       print(icd_code[idx],"Hit")
   else:
       print(icd_code[idx],"Miss")